In [16]:
import pandas as pd

In [17]:
data = pd.read_csv('data/KS_MRFN_AGE_ACCTO_RECOMMEND_SPORTS_INFO_202303.csv')

In [18]:
def weight_mapping(weight_text):
    if weight_text == "저체중":
        return -2.5
    elif weight_text == "정상":
        return 0
    elif weight_text == "비만전단계비만":
        return 1
    elif weight_text == "1단계비만":
        return 2
    elif weight_text == "2단계비만":
        return 3
    elif weight_text == "3단계비만":
        return 4

In [19]:
def sex_mapping(sex):
    if sex == "F":
        return -1
    if sex == "M":
        return 1

In [20]:
header = {}
for i in (list(data['RECOMEND_MVM_NM'])):
    
    if ( "동적" in i): continue
    if ( "스트레칭" in i): continue
    
    if i not in header:
        header[i] = 0
    
    header[i] += 1

In [21]:
# Sort the dictionary by value
header = dict(sorted(header.items(), key=lambda item: item[1], reverse=True))

# Get the top 10 sports
top_10_sports = list(header.keys())[:10]

filtered_data = data[data['RECOMEND_MVM_NM'].isin(top_10_sports)]

In [22]:
filtered_data

,AGRDE_FLAG_NM,BMI_IDEX_GRAD_NM,MBER_SEXDSTN_FLAG_CD,COAW_FLAG_NM,SPORTS_STEP_NM,FLAG_ACCTO_RECOMEND_MVM_RANK_CO,RECOMEND_MVM_NM
6,10대,정상,F,참가증,준비운동,2,몸통 비틀기
21,10대,정상,F,1등급,준비운동,2,몸통 비틀기
36,10대,정상,F,2등급,준비운동,2,몸통 비틀기
51,10대,정상,F,3등급,준비운동,2,몸통 비틀기
67,10대,정상,M,참가증,준비운동,3,몸통 비틀기
...,...,...,...,...,...,...,...
4998,70대 이상,3단계비만,M,1등급,본운동,4,엎드려 양팔 및 다리 들어올리기
5010,70대 이상,3단계비만,M,2등급,본운동,1,몸통 들어올리기
5014,70대 이상,3단계비만,M,2등급,본운동,5,엎드려 양팔 및 다리 들어올리기
5025,70대 이상,3단계비만,M,3등급,본운동,1,몸통 들어올리기


In [23]:
top_10_sports

['실내 자전거타기',
 '달리기',
 '다리 벌려 앞으로 상체 숙이기',
 '앉았다 일어서기',
 '버피운동',
 '몸통 비틀기',
 '다리 벌려 옆으로 상체 숙이기',
 '발목 얹고 다리 잡아당기기',
 '엎드려 양팔 및 다리 들어올리기',
 '몸통 들어올리기']

In [24]:
'''[
 '실내 자전거타기',
 '달리기',
 '다리 벌려 앞으로 상체 숙이기',
 '앉았다 일어서기',
 '버피운동',
 '몸통 비틀기',
 '다리 벌려 옆으로 상체 숙이기',
 '발목 얹고 다리 잡아당기기',
 '엎드려 양팔 및 다리 들어올리기',
 '몸통 들어올리기'
 ]'''
 
def one_hot_encoder(exercise:str):
    if exercise == "실내 자전거타기":
        return [1,0,0,0,0,0,0,0,0,0]
    elif exercise == "달리기":
        return [0,1,0,0,0,0,0,0,0,0]
    elif exercise == "다리 벌려 앞으로 상체 숙이기":
        return [0,0,1,0,0,0,0,0,0,0]
    elif exercise == "앉았다 일어서기":
        return [0,0,0,1,0,0,0,0,0,0]
    elif exercise == "버피운동":
        return [0,0,0,0,1,0,0,0,0,0]
    elif exercise == "몸통 비틀기":
        return [0,0,0,0,0,1,0,0,0,0]
    elif exercise == "다리 벌려 옆으로 상체 숙이기":
        return [0,0,0,0,0,0,1,0,0,0]
    elif exercise == "발목 얹고 다리 잡아당기기":
        return [0,0,0,0,0,0,0,1,0,0]
    elif exercise == "엎드려 양팔 및 다리 들어올리기":
        return [0,0,0,0,0,0,0,0,1,0]
    elif exercise == "몸통 들어올리기":
        return [0,0,0,0,0,0,0,0,0,1]

    return [0,0,0,0,0,0,0,0,0,0]

def one_hot_decoder(result):
    if result == 1:
        return "실내 자전거타기"
    
    if result == 2:
        return "달리기"
    
    if result == 3:
        return "다리 벌려 앞으로 상체 숙이기"
    
    if result == 4:
        return "앉았다 일어서기"
    
    if result == 5:
        return "버피운동"
    
    if result == 6:
        return "몸통 비틀기"
    
    if result == 7:
        return "다리 벌려 옆으로 상체 숙이기"
    
    if result == 8:
        return "발목 얹고 다리 잡아당기기"
    
    if result == 9:
        return "엎드려 양팔 및 다리 들어올리기"
    
    if result == 10:
        return "몸통 들어올리기"
    

In [25]:
import torch

trainset = []

for index, row in filtered_data.iterrows():
    
    trainset.append({
        'data': torch.tensor([
            sex_mapping(row["MBER_SEXDSTN_FLAG_CD"]),
            int(row['AGRDE_FLAG_NM'].replace("대","").replace(" 이상","")),
            weight_mapping(row['BMI_IDEX_GRAD_NM']),
        ]).float().to("cpu"),
        'target': torch.tensor(one_hot_encoder(row['RECOMEND_MVM_NM'])).float().to("cpu")
    })
    
        

In [26]:
import torch
import torch.nn as nn

In [27]:
class DNN(nn.Module):
    
    def __init__(self):
        super(DNN, self).__init__()
        
        self.fc1 = nn.Linear(3, 40)
        self.activation = nn.LeakyReLU()
        self.fc2 = nn.Linear(40, 10)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

In [28]:
import random

model = DNN()

# Move to GPU
model = model.to("cpu")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

epoch = 200

for i in range(epoch):
    
    # Reset the gradient
    optimizer.zero_grad()
    
    for this in random.sample(trainset, 100):
        output = model(this['data'])
        loss = criterion(output, this['target'])
        loss.backward()
        
    optimizer.step()    
    print(f"Epoch {i+1}/{epoch} Loss: {loss.item()}")

Epoch 1/200 Loss: 2.450568199157715
Epoch 2/200 Loss: 2.4576611518859863
Epoch 3/200 Loss: 2.447147846221924
Epoch 4/200 Loss: 2.3948259353637695
Epoch 5/200 Loss: 2.4238290786743164
Epoch 6/200 Loss: 2.454710006713867
Epoch 7/200 Loss: 2.438900947570801
Epoch 8/200 Loss: 2.4134368896484375
Epoch 9/200 Loss: 2.4239301681518555
Epoch 10/200 Loss: 2.285597085952759
Epoch 11/200 Loss: 2.2929701805114746
Epoch 12/200 Loss: 2.448291301727295
Epoch 13/200 Loss: 2.4220352172851562
Epoch 14/200 Loss: 2.424551248550415
Epoch 15/200 Loss: 2.4280271530151367
Epoch 16/200 Loss: 2.44999623298645
Epoch 17/200 Loss: 1.8342286348342896
Epoch 18/200 Loss: 2.440199851989746
Epoch 19/200 Loss: 2.446322202682495
Epoch 20/200 Loss: 2.4266276359558105
Epoch 21/200 Loss: 2.433307647705078
Epoch 22/200 Loss: 1.891574740409851
Epoch 23/200 Loss: 2.434197425842285
Epoch 24/200 Loss: 2.4257960319519043
Epoch 25/200 Loss: 2.431765079498291
Epoch 26/200 Loss: 1.8198254108428955
Epoch 27/200 Loss: 2.272146463394165

In [29]:
model.eval()

# Save model
#torch.save(model.state_dict(), "model.pth")
torch.save(model, "model.pt")

In [30]:
model.eval()

# Test the model
correct = 0
total = 0
one_hot_decoder(model(torch.tensor([1, 30, 0]).float().cuda()).argmax().item())

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking argument for argument mat2 in method wrapper_CUDA_mm)